In [ ]:
pip install deap

In [1]:
import random
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt

In [2]:
# Read the dataset
data = pd.read_csv('edge_device_data.csv')
print("Data loaded.")

Data loaded.


In [3]:
# Define the objective functions for performance and energy consumption
def evaluate(individual):
    cpu_usage, memory_usage, energy_consumption, latency, task_count, queue_length = individual
    
    # Normalize and invert the objectives for minimization
    normalized_cpu_usage = cpu_usage / max(data['CPU Usage (%)'])
    normalized_memory_usage = memory_usage / max(data['Memory Usage (MB)'])
    normalized_energy_consumption = energy_consumption / max(data['Energy Consumption (Wh)'])
    normalized_latency = latency / max(data['Latency (ms)'])
    normalized_task_count = task_count / max(data['Task Count'])  # To maximize

    # Inverting task_count for minimization (negative for maximization)
    return (normalized_cpu_usage, normalized_memory_usage, normalized_energy_consumption, normalized_latency, -normalized_task_count)

# Set up the DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0, -1.0, -1.0, 1.0))  # Minimizing all except task count
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, 100)  # Assuming the range for normalization
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=6)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)

# Adjusted mutation function to ensure it works with floats correctly
def mutPolynomialBoundedReal(individual, eta, lower_bound, upper_bound, indpb):
    size = len(individual)
    for i in range(size):
        if random.random() <= indpb:
            current_gene_value = individual[i]  # Current gene value
            gene_lower_bound, gene_upper_bound = lower_bound, upper_bound  # Lower and upper bounds
            delta1 = (current_gene_value - gene_lower_bound) / (gene_upper_bound - gene_lower_bound)  # Calculate change rate
            delta2 = (gene_upper_bound - current_gene_value) / (gene_upper_bound - gene_lower_bound)  # Calculate change rate
            rand = random.random()
            mutation_power = 1.0 / (eta + 1.0)  # Mutation power
            if rand < 0.5:
                gene_change_amount = 1.0 - delta1  # Change amount
                val = 2.0 * rand + (1.0 - 2.0 * rand) * (gene_change_amount ** (eta + 1.0))
                delta_q = val ** mutation_power - 1.0
            else:
                gene_change_amount = 1.0 - delta2  # Change amount
                val = 2.0 * (1.0 - rand) + 2.0 * (rand - 0.5) * (gene_change_amount ** (eta + 1.0))
                delta_q = 1.0 - val ** mutation_power
            current_gene_value = current_gene_value + delta_q * (gene_upper_bound - gene_lower_bound)  # Calculate new gene value
            # Ensure current_gene_value is a real number and within bounds
            current_gene_value = float(min(max(current_gene_value.real, gene_lower_bound), gene_upper_bound))
            individual[i] = current_gene_value  # Assign new gene value to individual
    return individual,

toolbox.register("mutate", mutPolynomialBoundedReal, lower_bound=0.0, upper_bound=100.0, eta=20.0, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Initialize the population
population = toolbox.population(n=100)

# Define the algorithm parameters
CXPB, MUTPB, NGEN = 0.6, 0.1, 50  # Adjusted crossover and mutation probabilities and number of generations

# Use the built-in DEAP algorithm for the evolutionary process
result_pop, log = algorithms.eaMuPlusLambda(
    population, toolbox, mu=200, lambda_=400, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=None, halloffame=None, verbose=True
)

# Extract the Pareto front
pareto_front = tools.sortNondominated(result_pop, len(result_pop), first_front_only=True)[0]

# Post-processing: Extracting the solutions
best_solutions = sorted(pareto_front, key=lambda ind: (ind.fitness.values[0], ind.fitness.values[1], ind.fitness.values[2], ind.fitness.values[3], -ind.fitness.values[4]))

# Collect the best solutions in a list
solutions = [individual for individual in best_solutions[:10]]

# Print the solutions
for solution in solutions:
    print(f"Solution: {solution}, Fitness: {solution.fitness.values}")

# Convert the fitness values to a list for further processing
fitness_values = [ind.fitness.values for ind in best_solutions]

gen	nevals
0  	100   
1  	267   
2  	258   
3  	275   
4  	305   
5  	283   
6  	289   
7  	287   
8  	285   
9  	266   
10 	288   
11 	282   
12 	257   
13 	284   
14 	280   
15 	272   
16 	296   
17 	277   
18 	271   
19 	285   
20 	283   
21 	273   
22 	277   
23 	273   
24 	259   
25 	278   
26 	289   
27 	281   
28 	279   
29 	277   
30 	265   
31 	271   
32 	281   
33 	267   
34 	289   
35 	269   
36 	275   
37 	281   
38 	253   
39 	283   
40 	281   
41 	281   
42 	286   
43 	284   
44 	291   
45 	275   
46 	284   
47 	278   
48 	296   
49 	281   
50 	271   
Solution: [-455.4928078669611, 9.343229141526496, 34.1056690365655, 56.5509436082849, 56.867161232712775, 76.34380300150774], Fitness: (-5.084334205532805, 0.004690769158010392, 6.821319068075701, 1.136495815299995, -1.960936594231475)
Solution: [-455.31271069463526, 52.09151889053316, 26.034698697343615, 57.83216198621233, 60.09829828531184, 63.47321806298315], Fitness: (-5.082323912070952, 0.026152552453156133, 5.207081159

In [4]:
# Evaluate using performance metrics
# Calculate Hypervolume
reference_point = np.max(fitness_values, axis=0) + 1
def calculate_hypervolume(solutions, ref_point):
    volumes = []
    for solution in solutions:
        volume = np.prod(ref_point - np.array(solution))
        volumes.append(volume)
    return np.sum(volumes)

hypervolume = calculate_hypervolume(fitness_values, reference_point)
print(f"Hypervolume: {hypervolume}")

# Calculate Spacing
def calculate_spacing(solutions):
    distances = []
    for i in range(len(solutions) - 1):
        dist = np.linalg.norm(np.array(solutions[i]) - np.array(solutions[i + 1]))
        distances.append(dist)
    return np.std(distances)

spacing = calculate_spacing(fitness_values)
print(f"Spacing: {spacing}")

# Calculate Pareto Front Spread
def calculate_pareto_front_spread(fitness_values):
    distances = [np.linalg.norm(np.array(f1) - np.array(f2)) for f1, f2 in zip(fitness_values[:-1], fitness_values[1:])]
    return np.sum(distances)

spread = calculate_pareto_front_spread(fitness_values)
print(f"Pareto Front Spread: {spread}")

Hypervolume: 3877.7102744503763
Spacing: 2.9658622371897394
Pareto Front Spread: 111.88487161723342


In [5]:
# Perform multiple runs to collect statistics
num_runs = 10
hypervolumes = []
spacings = []
pareto_spreads = []

for _ in range(num_runs):
    # DEAP algorithm for the evolutionary process
    result_pop, log = algorithms.eaMuPlusLambda(
        population, toolbox, mu=200, lambda_=400, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=None, halloffame=None, verbose=False
    )
    
    # Extract the Pareto front
    pareto_front = tools.sortNondominated(result_pop, len(result_pop), first_front_only=True)[0]
    
    # Convert the fitness values to a list for further processing
    fitness_values = [ind.fitness.values for ind in pareto_front]
    
    # Calculate performance metrics
    reference_point = np.max(fitness_values, axis=0) + 1
    hypervolume = calculate_hypervolume(fitness_values, reference_point)
    spacing = calculate_spacing(fitness_values)
    pareto_spread = calculate_pareto_front_spread(fitness_values)
    
    # Append results to lists
    hypervolumes.append(hypervolume)
    spacings.append(spacing)
    pareto_spreads.append(pareto_spread)

# Calculate mean and standard deviation for each metric
hypervolume_mean = np.mean(hypervolumes)
hypervolume_std = np.std(hypervolumes)
spacing_mean = np.mean(spacings)
spacing_std = np.std(spacings)
pareto_spread_mean = np.mean(pareto_spreads)
pareto_spread_std = np.std(pareto_spreads)

print(f"Hypervolume: Mean={hypervolume_mean}, Std={hypervolume_std}")
print(f"Spacing: Mean={spacing_mean}, Std={spacing_std}")
print(f"Pareto Front Spread: Mean={pareto_spread_mean}, Std={pareto_spread_std}")

Hypervolume: Mean=5607.135623338798, Std=8675.0306474224
Spacing: Mean=3.890298532315171, Std=2.6921751096711284
Pareto Front Spread: Mean=191.31198249216555, Std=138.61573711763754
